<a href="https://colab.research.google.com/github/RyutaKremer/A-star_with_Random-Pruning/blob/main/A-star_with_Random-Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import queue
import random
import numpy as np
import time

from PIL import Image

Base A* implementation

In [ ]:
def a_star_with_random_pruning(start_state, heuristic_func, expand_func, goal_test, 
                               weight=1, ranodom_ratio=None):
  g_scores = dict()
  open_list = queue.PriorityQueue()
  closed_list = set()
  rand_generator = random.Random(42)

  g_scores[start_state] = 0
  open_list.put((heuristic_func(start_state), start_state))

  stat = {}
  stat['time'] = 0  # Execution time
  stat['total expansion'] = 0
  stat['memory size'] = 0
  stat['score'] = 0

  # start taking time
  search_start = time.time()

  while not open_list.empty():
    best_value, best_state = open_list.get()

    if goal_test(best_state):
      search_end = time.time()
      stat['time'] = search_end - search_start
      stat['memory size'] = len(g_scores)
      stat['score'] = best_value
      return best_value, stat
    elif time.time() - search_start > 180:
      stat['time'] = '>180'
      stat['memory size'] = len(g_scores)
      stat['score'] = best_value
      return best_value, stat
    
    if best_state in closed_list:
      continue
    closed_list.add(best_state)

    for new_state, cost_best_to_new in expand_func(best_state, ranodom_ratio, rand_generator):
      stat['total expansion'] += 1

      g_new = g_scores[best_state] + cost_best_to_new
      if new_state in g_scores: # OPEN U CLOSES
        if g_scores[new_state] <= g_new:
          continue
        else:
          # new_state shouldn't removed in OPEN since 
          closed_list.discard(new_state)
      g_scores[new_state] = g_new
      open_list.put((g_scores[new_state] + heuristic_func(new_state) * weight, new_state))

  search_end = time.time()
  stat['time'] = search_end - search_start
  stat['memory size'] = len(g_scores)
  stat['score'] = -1
  return -1, stat

Functions for seach in maze

In [ ]:
def manhattan_distance_to_goal(state):
  x, y = state
  gx, gy = goal_state
  return abs(x-gx) + abs(y-gy)

In [ ]:
def number_of_separating_walls(state):
  x, y = state
  gx, gy = goal_state
  return np.sum(maze_array[x:gx] == 0) + np.sum(maze_array[y:gy] == 0)

In [ ]:
def expand_in_maze_4d(state, ranodom_ratio, rand_generator):
  x, y = state
  dx = [1, 0, -1, 0]
  dy = [0, 1, 0, -1]
  r_states = []

  for i in range(4):
    if ranodom_ratio is not None:
        if rand_generator.uniform(0, 1) < ranodom_ratio:
          continue  # skip new node
    n_state_x = x + dx[i]
    n_state_y = y + dy[i]
    if 0 <= n_state_x < w and 0 <= n_state_y < l and maze_array[n_state_x, n_state_y] != 0:
      r_states.append(((n_state_x, n_state_y), 1))
    
  return r_states

In [ ]:
def expand_in_maze_8d(state, ranodom_ratio, rand_generator):
  x, y = state
  dx = [1, 1, 0, -1, -1, -1, 0, 1]
  dy = [0, 1, 1, 1, 0, -1, -1, -1]
  r_states = []

  for i in range(8):
    if ranodom_ratio is not None:
        if rand_generator.uniform(0, 1) < ranodom_ratio:
          continue
    n_state_x = x + dx[i]
    n_state_y = y + dy[i]
    if 0 <= n_state_x < w and 0 <= n_state_y < l and maze_array[n_state_x, n_state_y] != 0:
      r_states.append(((n_state_x, n_state_y), 1))
    
  return r_states

In [ ]:
def goal_test(state):
  return state == goal_state

Experiments for maze

In [ ]:
# Chose maze here
# image = Image.open("maze1.png")
image = Image.open("maze2.png")

In [ ]:
threshold = 200

image = image.convert("L").point(lambda x: 0 if x < threshold else 1, mode="1")

array = np.array(image)
maze_array = np.array([row[::2] for row in array[::2]]).astype(int)
# np.savetxt("maze.txt", maze_array, fmt="%d", delimiter="")

print(maze_array)

w, l = maze_array.shape
print(w, l)
start_state = (0, (w+1)//2 - 5)
goal_state = (l-1, (w+1)//2 + 3)

maze_array_to_show = maze_array.astype(str)
maze_array_to_show[start_state] = 'S'
maze_array_to_show[goal_state] = 'G'
np.savetxt("maze.txt", maze_array_to_show, fmt="%s", delimiter="")

In [ ]:
ratio = [None, 0, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5]
ratio_name = ['None', '0% (With overhead)', '1%', '2%', '3%', '5%', '10%', '20%', '30%', '50%']

In [ ]:
df = pd.DataFrame()
for weight, algo_name in zip([1, 0.5, 2], ['A*', 'WA* (w=0.5)', 'WA* (w=2)']):
  for hf in ['manhattan distance', 'separating wall']:
    for expand_dir in [4, 8]:
      for r, rn in zip(ratio, ratio_name):
        heuristic_func = manhattan_distance_to_goal if hf == 'manhattan distance' \
          else number_of_separating_walls
        expand_func = expand_in_maze_4d if expand_dir == 4 else expand_in_maze_8d
        _, stat = a_star_with_random_pruning(start_state, heuristic_func, expand_func, goal_test,
                                             weight=weight, ranodom_ratio=r)
        stat['Algorithm'] = algo_name
        stat['Heuristics'] = hf
        stat['Branch'] = expand_dir
        stat['Random ratio'] = rn
        print(stat)
        df = df.append(stat, ignore_index=True)
df.to_csv('Result.csv')
df